## NER Training Preparation
- Download base `config` file from https://spacy.io/usage/training#quickstart
- Modify config file for training data
- Prepare training data
- Do training
- Final testing

Data
- https://github.com/Alwyn25/Resumeparsing.git

## Data Annotation Tools


- http://agateteam.org/spacynerannotate/

In [1]:
!%pip install spacy_transformers
%pip install -U spacy

/bin/bash: line 0: fg: no job control
  Using cached spacy-3.5.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached thinc-8.1.7-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (828 kB)
  Using cached srsly-2.4.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (492 kB)
  Using cached blis-0.7.9-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.2 MB)
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.2
    Uninstalling catalogue-1.0.2:
      Successfully uninstalled catalogue-1.0.2
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.6
    Uninstalling srsly-1.0.6:
      Successfully uninstalled srsly-1.0.6
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.6
   

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

import warnings
warnings.filterwarnings('ignore')

In [4]:
spacy.__version__

'3.5.0'

In [5]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [7]:
!git clone https://github.com/Alwyn25/Resumeparsing.git

Cloning into 'Resumeparsing'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 27 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (27/27), 5.30 MiB | 1.37 MiB/s, done.


In [11]:
cv_data = json.load(open('content/Resumeparsing/data/training/train_data.json', 'r'))

In [12]:
len(cv_data)

200

In [29]:
!python -m spacy init fill-config /content/Resumeparsing/data/training/base_config.cfg /content/Resumeparsing/data/training/config.cfg

In [14]:
# cv_data[0]

In [18]:
def get_spacy_doc(file, data):
    nlp = spacy.blank('en')
    db = DocBin()

    for text, annot in tqdm(data):
        doc = nlp.make_doc(text)
        annot = annot['entities']

        ents = []
        entity_indices = []

        for start, end, label in annot:
            skip_entity = False
            for idx in range(start, end):
                if idx in entity_indices:
                    skip_entity=True
                    break
            if skip_entity==True:
                continue

            entity_indices = entity_indices + list(range(start, end))

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                err_data = str([start, end]) + "    " + str(text) + "\n"
                file.write(err_data)

            else:
                ents.append(span)

        try:
            doc.ents = ents
            db.add(doc)
        except:
              pass

    return db
      

In [19]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [20]:
len(train), len(test)

(140, 60)

In [26]:
file = open('error.txt', 'w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

In [27]:
len(db.tokens)

In [ ]:
!python -m spacy train /content/Resumeparsing/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-06-17 20:10:05,414] [INFO] Set up nlp object from config
[2022-06-17 20:10:05,424] [INFO] Pipeline: ['transformer', 'ner']
[2022-06-17 20:10:05,428] [INFO] Created vocabulary
[2022-06-17 20:10:05,429] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a 

### Model Test

In [ ]:
nlp = spacy.load('/content/output/model-best')


In [ ]:
doc = nlp('my name is Laxmi Kant Tiwari. I worked at Microsoft. I have 10 years of experience')
for ent in doc.ents:
    print(ent.text, "   ->>>>> ", ent.label_)

Laxmi Kant    ->>>>>  Name
Microsoft    ->>>>>  Companies worked at
10 years    ->>>>>  Years of Experience


In [1]:
%pip install PyMuPDF

Defaulting to user installation because normal site-packages is not writeable
  Using cached PyMuPDF-1.21.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sys, fitz

In [ ]:
fname = '/content/Resumeparsing/data/test/Alice Clark CV.pdf'
doc = fitz.open(fname)

In [ ]:
# doc = [page.getText() for page in doc]

In [ ]:
text = " "
for page in doc:
    text = text + str(page.get_text())

In [ ]:
text = text.strip()

In [ ]:
text = ' '.join(text.split())

In [ ]:
text

'Alice Clark AI / Machine Learning Delhi, India Email me on Indeed • 20+ years of experience in data handling, design, and development • Data Warehouse: Data analysis, star/snow flake scema data modelling and design specific to data warehousing and business intelligence • Database: Experience in database designing, scalability, back-up and recovery, writing and optimizing SQL code and Stored Procedures, creating functions, views, triggers and indexes. Cloud platform: Worked on Microsoft Azure cloud services like Document DB, SQL Azure, Stream Analytics, Event hub, Power BI, Web Job, Web App, Power BI, Azure data lake analytics(U-SQL) Willing to relocate anywhere WORK EXPERIENCE Software Engineer Microsoft – Bangalore, Karnataka January 2000 to Present 1. Microsoft Rewards Live dashboards: Description: - Microsoft rewards is loyalty program that rewards Users for browsing and shopping online. Microsoft Rewards members can earn points when searching with Bing, browsing with Microsoft Edg

In [ ]:
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, "   ->>>>> ", ent.label_)

Alice Clark    ->>>>>  Name
AI / Machine Learning    ->>>>>  Degree
Delhi    ->>>>>  Location
• 20    ->>>>>  Degree
• Data    ->>>>>  Degree
• Database    ->>>>>  Degree
Microsoft    ->>>>>  Companies worked at
Software Engineer    ->>>>>  Designation
Microsoft    ->>>>>  Companies worked at
– Bangalore,    ->>>>>  Degree
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Microsoft    ->>>>>  Companies worked at
Indian Institute of Technology    ->>>>>  College Name
– Mumbai    ->>>>>  Degree
Machine Learning, Natural Language Processing, and Big Data Handling ADDITIONAL INFORMATION Professional Skills • Excellent analytical, problem solving, communication, knowledge transfer and interpersonal skills with ability to interact with individuals at all the levels • Quick learner and maintains cordial relationship with project manager and